以下是Coursera上的[How to Win a Data Science Competition: Learn from Top Kagglers](https://www.coursera.org/learn/competitive-data-science/home/week/3)课程笔记。

# Hyperparameter Optimization
- List most important hyperparameters in major models; describe their impact
- Understand the hyperparameter tuning process in general
- Arrange hyperparameters by their importance

## Hyperparameter tuning I
**Plan for the lecture**
- Hyperparameter tuning in general
 - General pipeline
 - Manual and automatic tuning
 - What should we understand about hyperparameters?
- Models,libraries and hyperparameter optimization
 - Tree-based models
 - Neural networks
 - Linear models
 
**Plan for the lecture:models**
- Tree-based models
 - GBDT: XGBoost, LightGBM, CatBoost
 - RandomForest/ExtraTrees
- Neural nets
 - Pytorch, Tensorflow, Keras...
- Linear models
 - SVM, logistic regression
 - Vowpal Wabbit, FTRL
- *Factorization Machines(out of scope)*
 - libFM, libFFM

### How do we tune hyperparameters
- 1.Select the most influential parameters
 - a.There are tons of parameters and we can'ttune all of them
- 2.Understand,how exactly they influence the training
- 3.Tune them
 - a.Manually(change and examine)
 - b.Automatically(hyperopt, etc)

- 1.
无论如何，我们从来没有时间调整所有的参数，所以我们需要提出一个很好的子集来调整。假设我们是xgboost新手，不知道哪些参数是需要调的，可以在Github或Kaggle Kernels搜索到前人通常设置的参数。
- 2.
理解改变其中一个参数会发生什么。
- 3.
大多数人手动完成调参工作。也可以使用超参数优化工具，但手动执行通常会更快。


### Hyperparameter optimization software自动调参工具
运行调参工具可能需要很长时间，因此最好的策略是在夜间运行它。
- A lot of libraries to try:
 - Hyperopt
 - Scikit-optimize
 - Spearmint
 - GPyOpt
 - RoBO
 - SMAC3

从广义上讲，不同的参数会导致三种不同的结果
- 1.Underfitting(bad)
- 2.Good fit and generalization(good)
- 3.Overfitting(bad)

因此我们需要把想要调整的参数分为两组。第一组是约束模型的参数，第二组与第一组效果相反。
- **A parameter in red**
 - Increasing it impedes fitting
 - Increase it to reduce overfitting
 - Decrease to allow model fit easier
- **A parameter in green**
 - Increasing it leads to a batter fit(overfit) on train set
 - Increase it, if model underfits
 - Decrease if overfits

*上面提到的颜色只是视频中的标记*

## Hyperparameter tuning II
一些基于树模型的超参数优化
- Tree-based models

|Model|Where|
|:-:|:-:|
|GBDT|XGBoost-[dmlc/xgboost](https://github.com/dmlc/xgboost)<br>LightGBM-[Microsoft/LightGBM](https://github.com/Microsoft/LightGBM)<br>CatBoost-[catboost/catboost](https://github.com/catboost/catboost)|
|RandomForest/ExtraTrees|*scikit-learn*|
|Others|RGF-[baidu/fast_rgf](https://github.com/baidu/fast_rgf)|

### GBDT

|XGBoost|LightGBM|
|:-:|:-:|
|max_depth|max_depth/num_leaves|
|subsample|bagging_fraction|
|colsample_bytree,<br>colsample_bylevel|frature_fraction|
|*`min_child_weight,`<br>`lambda,alpha`*|*`min_data_in_leaf,`<br>`lambda_l1,lambda_l2`*|
|eta<br>num_round|learning_rate<br>num_iterations|
|Others:<br>seed|Others:<br>*_seed|


- **max_depth**:<br>树越深，越能拟合数据集，但这可以会导致过拟合。根据任务的不同，最大深度可能会有很大差异，有时是2，有时是27。建议max_depth大约从7开始，直到未过拟合的最大深度。需要注意的是深度增加，学习时间就更长。
 - **num_leaves**:<br>在LightGBM中，可以控制叶的数量，而不是最大深度。因为树可以很深，但如果叶子数量少就不会导致过拟合。
- **subsample、bagging_fraction**:<br>这个参数可以控制每次喂给模型的数据量，取值在0,1之间。每次喂给它一小部分数据，可以让它不那么过拟合，并且可以得到更好的泛化效果，但是模型的训练会更慢。这有点像正则化的作用。
- **colsample_bytree、colsample_bylevel**:<br>这个参数可以控制subsample中的分裂点。如果模型过拟合，可以尝试降低这些值。
- **min_child_weight,lambda,alpha**:<br>正则化参数。
 - **min_child_weight**:<br>经验中，这是最重要的参数。增加它可以让模型更保守，减少它会让模型有更少约束。根据不同的任务，我发现最佳值为0,5,15,300，所以不要犹豫，尝试各种值，这取决于数据。
- **eta、num_round**:eta本质上是一种学习权重，就像梯度下降一样。num_round是我们想要执行的学习步数，换句话说，是我们想要建多少棵树。每次迭代都会构建一个新树，以学习率eta添加到模型中。
 - 当我们找到合适的轮数时，可以做一个通常会提高分数的技巧。我们将num_round乘以α，将eta除以α，模型通常会变得更好。可能应用的参数也需要调整，但通常可以保留原样。
 
 **Other**
 - **seed**:<br>一般情况下随机种子对于模型影响不大。但如果随机种子对你的影响非常大时，建议你可以多次提交，或者根据随机性调整你的验证方案。

### sklearn.RandomForest/ExtraTrees

- **n_estimators**:<br>RandomForest构建每棵树是独立于其他树的，这意味这拥有大量树的模型不会导致过拟合，这于Gradient Boosting相反。我们通常首先将n_estimators设置为非常小的数字，例如10，并看看这将花费多少时间，如果不太长，就把它设为一个比较大的值，例如300。
- **max_deep**:<br>控制树的深度，于XGBoost不同，它可以被设置为None,这对应于无限深度。当数据集中的特征具有重复值和重要交互时，它实际上非常有用。在其他情况下，无约束深度的模型将立即过拟合。建议随机森林的深度从7左右开始。通常随机深林的最佳深度高于Gradient Boosting，所有不要犹豫尝试10,20或更高的值。
- **max_feature**:<br>与XGBoost中的参数相同。
- **min_samples_leaf**:<br>是一个类似正则化的参数，与XGBoost的min_child_weight和LightGBM的min_data_leaf相同。

**Other**
- **criterion**:<br>根据我的经验，Gini更常见，但有时Entropy更好。
- **random_state**:<br>随机种子参数
- **n_jobs**:设置拥有多个核心数。默认情况下sklearn的RandomForest由于某种原因仅使用一个核心。

## Hyperparameter tuning III
- Neural nets
 - Pytorch, Tensorflow, Keras...
- Linear models
 - SVM, logistic regression
 - Vowpal Wabbit, FTRL

### Neural Nets
这里讨论的是dense neural nets,即只含有全连接层的网络

自适应算法已高亮+斜体显示
- Number of neurons per layer
- Number of layers
- Optimizers
 - *`SGD + momentum`*
 - Adam/Adadelta/Adagrade/..
   - In pratice lead to more overfitting
- Batch size
- Learning rate
- Regularization
 - L2/L1 for weights
 - Dropout/Dropconnect
 - Static Dropconect

- 建议从简单的开始，比如1层或2层，调试代码，确保训练时loss下降
- 然后尝试找到一个能够过拟合的配置，之后在网络中调整一些东西
- 神经网络的关键部分之一是优化方法
 - 自适应优化方法的确可以让你更快的拟合数据，但根据我的经验，这也会导致严重的过拟合。普通的SGD收敛速度较慢，但是训练好的模型通常会有更好的泛化效果。Adaptive methods are useful,but in the settings others in classification and regression.
- Batch Size:事实证明批量过大会导致更多的过拟合。凭经验，batch_size为500就可以认为很大。建议选择32或64左右的值，如果网络仍然过拟合，请尝试减少batch_size，反之增加它。batch_size也不应该太小，否则梯度可能会有太多噪声。在调整batch_size后，必要时，应该去调整其他网络数量。
- 学习率：学习率不能太高也不能太低。因此，最佳学习率取决于其他参数。通常从一个大的学习率开始，比如0.1，然后逐步去减小它。有一条经验法则，如果你将batch_size增加alpha倍，你也可以提高学习率alpha倍。
- 早期，人们大多使用L2和L1正则化。如今大多数人都使用dropout正则化。对我来说，就是在数层之后立即将dropout作为第一层。
- static dropconnect:通常我们有一个密集连接的输入层，比如128个单位。我们将改为一个非常巨大的隐藏层，比如4096个单位，对于一般的比赛来说，这是一个巨大的网络，它会严重过拟合。现在为了规范它，我们将对这一层随机dropout 99%，这是非常强的正则化，实践证明这是可以的。
![dropconnect](img/dropconnect.png)


### Linear models
- **Scikit-learn**
 - SVC/SVR
   - Sklearn wraps `libLinear` and `libSVM`
   - Compile yourself for multicore support
 - LogisticRegression/LinearRegression + *regularizers*
 - SGDClassifier/SGDRegressor
 
 
- **Vowpal Wabbit**
 - FTRL

- SVM几乎不需要调参，这是最大的益处
- 最新版的`libLinear`和`libSVM`支持多核处理，但Sklearn中的不支持多核处理。所以我们需要动手变异这些库以使用此选项。
- 几乎没有人使用`kernel SVC`，所以这里只讨论SVM
- 对于不适合在内存中操作的数据，我们可以使用`Vowpal Wabbit`，它以在线的方式实现线性模型的学习。它只能直接从硬盘驱动器中逐行读取数据，永远不会将整个数据集加载到内存中。因此，允许学习非常庞大的数据集。
- 线性模型的在线学习方法（FTRL）在前段时间特别受欢迎，他是`Vowpal Wabbit`中的实现。

### Linear models
- Regularization parameter(X,alpha,lambda,..)
 - Start with very small value and increase it.
 - SVC starts to work sklowe as C increase
- Regularization type
 - L1/L2/L1+L2 --try each
 - L1 can be used for feature selection

- C：对于SVM，我通常会从一个非常小的值开始，比如$10^{-6}$，每次乘以10。从小的值开始，是因为参数C越大，训练时间越长。
- 选择L1还是L2？答案是尝试两者，在我看来，它们非常相识。并且L1还有一个好处，可以给我们提供一个稀疏权重，这可以用于特征选择。

## Tips
- **Don't spend too much time tuning hyperparameters**
 - Only if you don't have any more ideas or you have spare computational resources

- **Be patient**
 - It can take thousands of rounds for GBDT or neural nets to fit.
 
- **Average everything**
 - Over random seed
 - Or over small deviations from optimal parameters
   - e.g.average max_depth=4,5,6for an optimal 5

## 相关链接
- [调整估计器的超参数（sklearn）](http://scikit-learn.org/stable/modules/grid_search.html)
- [Python中梯度提升（GBM）中参数调整的完整指南](https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/)